In [ ]:
import os
import subprocess
import csv
import re
import sys
import xml.etree.ElementTree as ET
from PIL import Image
import shutil

In [ ]:


def extract_timestamps_from_sup(sup_file, output_dir):
    """Extract XML from a SUP file using BDSup2Sub."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Command to extract XML using BDSup2Sub
    cmd = [
        'java', '-jar', '/home/coof/BDSup2Sub.jar',
        sup_file,
        '-o', os.path.join(output_dir, 'output.xml'),  # Output XML file
    ]
    
    # Run the command
    subprocess.run(cmd, check=True)


In [ ]:
def create_srt_entry(index, start_time, end_time):
    """Format timestamps into SRT entry."""
    return f"{index}\n{start_time} --> {end_time}\n\n"


In [ ]:
# specifically for srts already


def parse_timecode(timecode):
    """Convert timecode from format 'hh:mm:ss,ms' to 'hh:mm:ss:ff'."""
    hh, mm, ss_ms = timecode.split(':')
    ss, ms = ss_ms.split(',')
    ff = int(ms) // 40  # Convert milliseconds to frame count (assuming 25fps)
    return f'{hh}:{mm}:{ss}:{ff:02d}'

def process_subtitle_file(input_file, output_csv):
    with open(input_file, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Index", "Start Time", "End Time", "Text"])

        lines = infile.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()

            # Check if the line is a numeric index
            if line.isdigit():
                index = int(line)  # Capture the subtitle index
                i += 1

                # The next line should be the timestamp
                timestamp_line = lines[i].strip()
                start_time, end_time = timestamp_line.split(' --> ')
                start_time = parse_timecode(start_time.strip())
                end_time = parse_timecode(end_time.strip())
                i += 1

                # Collect all subtitle text lines until a blank line is encountered
                subtitle_text = []
                while i < len(lines) and lines[i].strip() != "":
                    subtitle_text.append(lines[i].strip())
                    i += 1

                # Write the collected data to the CSV
                writer.writerow([index, start_time, end_time, " ".join(subtitle_text)])

            i += 1

# # Example usage:
# process_subtitle_file('input.txt', 'output.csv')

In [ ]:
def convert_sup_to_csv(sup_file, output_csv, xml_file):
    # Directory to store the extracted XML
    output_dir = "temp"
    
    # Step 1: Extract timestamps from the SUP file
    extract_timestamps_from_sup(sup_file, output_dir)
    
    # Step 2: Parse the XML and extract timing information
    tree = ET.parse(os.path.join(output_dir, 'output.xml'))
    root = tree.getroot()

    shutil.copy(os.path.join(output_dir, 'output.xml'), xml_file)
    
    # Open the CSV file for writing
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write the header row
        csvwriter.writerow(['Index', 'Start Time', 'End Time'])
        
        index = 1
        
        for sub in root.findall('.//Event'):
            start_time = sub.get('InTC')
            end_time = sub.get('OutTC')
            
            # Write the row to the CSV file
            csvwriter.writerow([index, start_time, end_time])
            index += 1

In [ ]:
def process_subtitle_files(source_dir, destination_dir):
    # Define the range of seasons (e.g., Season 1 to Season 7)
    seasons = sorted([f"Season {i}" for i in range(5, 7)])

    # Loop through each season folder
    for season in seasons:
        season_path = os.path.join(source_dir, season)

        # Check if the season directory exists
        if not os.path.isdir(season_path):
            print(f"Directory {season_path} does not exist, skipping.")
            continue

        # Create corresponding destination directory for the season
        destination_season_path = os.path.join(destination_dir, season)
        os.makedirs(destination_season_path, exist_ok=True)

        # Get sorted list of files in the season folder
        files = sorted(os.listdir(season_path))

        # Process each file in the season folder
        for filename in files:
            # Look for complete .mkv files
            if filename.endswith('.mkv') and not filename.endswith('.mkv.part'):

                pattern = r"S\d{2}E\d{2} - [^()]+"

                match = re.search(pattern, filename)
                if match:
                    modified_filename = match.group(0)

                video_file = os.path.join(season_path, filename)
                sup_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.sup")
                csv_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.csv")
                xml_file = os.path.join(destination_season_path, modified_filename.strip(), "subtitles.xml")
                

                # convert_sup_to_csv(sup_file, csv_file, xml_file)
                # process_subtitle_file(sup_file, csv_file)
                # sys.exit()


In [ ]:
# Specify the source and destination directories
source_directory = "./Silicon Valley Raw"  # Replace with the path to your directory containing seasons
destination_directory = "./Silicon Valley"

# Call the function to burn subtitles
process_subtitle_files(source_directory, destination_directory)
